## 테스트 코드

In [1]:
import os
import pandas as pd
import openai
from openai import OpenAI
from scipy.spatial.distance import cosine
client = OpenAI()
openai.api_key = os.environ["OPENAI_API_KEY"]

In [14]:
df = pd.read_pickle("./job-processed2.pkl")
def create_context(
    question, df, max_len=3000
):
    q_embeddings = client.embeddings.create(input=question, model='text-embedding-ada-002').data[0].embedding
    df["distances"] = df["embedding"].apply(lambda x: cosine(q_embeddings, x))
    returns = []
    cur_len = 0
    for _, row in df.sort_values('distances', ascending=True).iterrows():
        cur_len += row['n_tokens'] + 4
        if cur_len > max_len:
            break
        returns.append(row["combined"])
    return "\n\n===\n\n".join(returns)


def answer_question(
    df,
    model="gpt-3.5-turbo",
    question="python을 할 줄 알면, 어떤 회사에서 일을 할 수 있어?",
    max_len=3000,
    debug=False,
):
    context = create_context(
        question,
        df,
        max_len=max_len,
    )
    if debug:
        print("Context:\n" + context)
        print("\n\n")

    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                # {"role": "system", "content": "Answer the question based on the context below, and if the question can't be answered based on the context, say \"I don't know\"\n\n"},
                # {"role": "user", "content": f"context: {context}\n\n---\n\n Question: {question}, 한국어로 대답해줘."}
                {"role": "system", "content": "You are a helpful assistant who offer job information. Answer the question in Korean using Answer Form replacing `` with words based on the context below, and if the question can't be answered based on the context, say \"I don't know\"\n\n"},
                {"role": "user", "content": f"Context: {context}\n\n---\n\n Quengstion: {question}."},
                {"role": "assistant", "content": """Answer Form: For example, `company A`, `company B` and `company C` are looking for people who can use `programing language`. Are there specific types of companies you're interested in?, `A` is a company established in `year` and at 'location`. `A` has `goals` and is a `adjective` company. `A` provides additional welfare benefits like `a, b and c`, For `job category`, languages like `language A`, `language B`, and `language C` are fundamental,commonly used,may be relevant., In general, companies expect you to do `a, b, and c` for `job A`., `A, B and C` are open to new hires., To work as a `job category`, you should know `a`, `b, ans `c` programming languages., The programming languages that the company is looking for are `a`, `b`, and `c`."""}
            ],
            temperature=0,
        )
        return response.choices[0].message.content
    except Exception as e:
        print("Error occurred:", e)
        return ""

In [13]:
# text-embedding-3-small
answer_question(df, question="If I can use Python, which companies can I apply to?")

'Python을 사용할 수 있다면, `에스엔에이`, `피데스어드바이저리`, `로앤굿`과 같은 회사에 지원할 수 있습니다. 특정 유형의 회사에 관심이 있으신가요? `에스엔에이`는 `년도`에 설립된 회사로 `위치`에 위치하고 있습니다. `에스엔에이`는 `목표`를 가지고 있으며, `형용사` 회사입니다. `에스엔에이`는 `a, b, c`와 같은 추가 복지 혜택을 제공합니다. `피데스어드바이저리`는 `a, b, c`와 같은 언어를 다루는 것이 중요한 `직종`에 대한 인재를 찾고 있습니다. 일반적으로 회사들은 `직종`에 대해 `a, b, c`를 수행할 수 있는 능력을 기대합니다. `로앤굿`은 새로운 인재를 모집하고 있습니다. `직종`으로 일하기 위해서는 `a, b, c`와 같은 프로그래밍 언어에 대한 지식이 필요합니다. 회사에서 요구하는 프로그래밍 언어는 `a, b, c`입니다.'

In [15]:
# text-embedding-ada-002
answer_question(df, question="If I can use Python, which companies can I apply to?")

'Python을 사용할 수 있다면, `어센트코리아`와 `슈어소프트테크`와 같은 회사에 지원할 수 있습니다. `어센트코리아`는 검색 데이터를 분석하여 마케팅에 활용하는 인텐트 마케팅 전문 기업이며, `슈어소프트테크`는 국방 소프트웨어 검증을 위한 테스트 자동화 도구를 개발하는 소프트웨어 품질 전문 기업입니다. 이 외에도 Python을 사용하는 다른 회사들도 있을 수 있습니다. 특정 분야의 회사에 관심이 있으신가요?'